# Damped Harmonic Oscillator 

Damped harmonic oscillator takes form:

$y'' + 2\gamma y' + \omega_0^2 y = 0$ and has general solution:

$y(t) = A \exp{\left(\left( -\gamma + \sqrt{\gamma^2 -\omega_0^2}\right)t\right)} + B \exp{\left(\left( -\gamma - \sqrt{\gamma^2 -\omega_0^2}\right)t\right)}$

Require $\omega_0^2 > \gamma^2$ for underdamped oscilation. For this scenario, the general solution becomes

$y(t) = C e^{- \gamma t} \left(\exp{i (\sqrt{\omega_0^2 - \gamma^2}) t} + \exp{i (\sqrt{\omega_0^2 - \gamma^2}) t}\right)\\$
$\Rightarrow y(t) = 2C e^{-\gamma t} \cos{\left(\omega t + \phi\right)} \\$

where $\omega = \sqrt{\omega^2_0 -\gamma^2}$ and $\phi$ is some phase offset determined by the initial conditions. We will take $\phi = 0$ so the oscillator is at maximum displacement at $t=0$. 

Aim to make a neural network to approximate the harmonic oscillator function, initialised with random weights and biases.

In [3]:
import torch as t
import numpy as np 
import torch.nn as nn
import torch.optim as optim 
import torch.nn.functional as F #contains various functions including activation functions 
import matplotlib.pyplot as plt 

In [59]:
#Analytical Solution
#A = 2C here

def analytical(time, gamma, omega_0, A):
    omega = np.sqrt(omega_0**2 - gamma**2)
    return A*np.exp(-gamma*time)*np.cos(omega*time)

#System parameters
gamma = 1
omega_0 = 2
A = 1

t_analytical = np.linspace(0, 1, 100)
y_analytical = analytical(t_analytical, gamma, omega_0, A)
t_initial = t.tensor(0., requires_grad=True).view(-1, 1) #t=0 tensor
with t.no_grad():
    print(t_initial)

tensor([[0.]], grad_fn=<ViewBackward0>)


In [35]:
class Model(nn.Module): #to call nn.Module to inherit pytorch nn functionality
    def __init__(self, in_channels, out_channels, hidden_channels, num_hidden_layers:int=2):
        #This function initialises the neural network, setting up the infrastructure to be used by forward
        #hidden_channels parameter is the number of neurons in each hidden layer
        super().__init__() #initialise nn.Module parent class first to use nn infrastructure
        self.input_layer = nn.Linear(in_channels, hidden_channels) #transforms input to match hidden
    
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_channels, hidden_channels) for _ in range(num_hidden_layers)]) 
        #creates {num_hidden_layers} hidden layers and adds a linear transformation 
        #with its own trainable weights and bias between each layer and ensures the inputs
        #and outputs of each hidden layer match the next.

        self.output_layer = nn.Linear(hidden_channels, out_channels) #transforms output of hidden layers
        #to the shape of the output channels

    def forward(self, x: t.tensor):
        #defines how data flows through the network 
        #x is a tensor with shape [batch_size, in_channels]
        x = F.tanh(self.input_layer(x)) #tanh activation function after first layer, introduces non-linearity
        #x leaves as tensor with shape [batch_size, hidden_channels]
        for hidden_layer in self.hidden_layers:
            x = F.tanh(hidden_layer(x)) #iterates through all hidden layers, applies linear transformation
            #and tanh activation
        #x leaves as tensor with shape [batch_size, output_channels]
        x = self.output_layer(x) #no activation function here
        return x 

Above code defines the neural network structure and the forward pass ie the neural network model. Key features are the requirement of inheriting the nn.Module class to make use of the pytorch neural network features off the bat- allows parameter tracking and easy gradient calculation. The number of hidden layers and neurons in each hidden layer can be tuned. Each layer applies a linear transformation to reshape its input into the correct shape to be passed on to the next layer, each transformation has its own weights and bias which can be trained. Use of super.__init__() allows layers to be looped through, indexed and their parameters can be accessed using list(model.parameters()). forward function adds tanh  activation functions after each linear transformation before being passed on to next layer.

In [60]:
#Next step is loss and optimiser
learning_rate = 0.01
model = Model(1, 1, 10)
with t.no_grad():
    for param in model.parameters():
        print(param.shape, param[0]) #randomised weights and biases
optimiser = optim.Adam(model.parameters(), lr = learning_rate) #pass optimiser the trainable parameters






# number_iterations = 1000
# for epoch in range(number_iterations):
y_predicted = model(t_initial)
loss_y_IC = (t.squeeze(y_predicted) - A)**2 #start at maximum amplitude, squeeze returns input without 
#specified dimensions of 1

dy_predicted = t.autograd.grad(y_predicted, t_initial, t.ones_like(y_predicted), create_graph=True)[0]
#above calculates the gradient of y_predicted wrt t_initial. t.ones_like gives vector of correct shape 
#for vector jacobian product. ones_like gives direction of derivative and sums each derivative (1*y_1 + ... + 1*y_n). 
#create_graph=True allows higher order derivatives to be created from this gradient object. 
loss_dy_IC = (t.squeeze(dy_predicted) - 0)**2 #start with zero velocity

d2y_predicted = t.autograd.grad(dy_predicted, t_initial, t.ones_like(dy_predicted))[0]

    




torch.Size([10, 1]) tensor([-0.4675], requires_grad=True)
torch.Size([10]) tensor(-0.2674, requires_grad=True)
torch.Size([10, 10]) tensor([-0.3005, -0.2397, -0.3107,  0.2610,  0.1412, -0.2743, -0.0058, -0.1953,
         0.2685,  0.1516], requires_grad=True)
torch.Size([10]) tensor(0.0939, requires_grad=True)
torch.Size([10, 10]) tensor([-0.2583, -0.1546, -0.1749,  0.0857,  0.0121,  0.2327, -0.3106, -0.2068,
         0.1159, -0.2713], requires_grad=True)
torch.Size([10]) tensor(0.0430, requires_grad=True)
torch.Size([1, 10]) tensor([ 0.1806,  0.0419,  0.0551, -0.2389,  0.0820,  0.0681, -0.2659, -0.2032,
        -0.2620, -0.1456], requires_grad=True)
torch.Size([1]) tensor(-0.0398, requires_grad=True)
